In [2]:
# Import useful library
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sklearn as sk
import pandas
import numpy as np
import tensorflow as tf
import random
import os
import collections
import math
import time
import datetime
from tensorflow.contrib import learn
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from matplotlib import pylab

# Allow print picture or graph in Ipython
%matplotlib inline
# Allow print all arry if command print called
np.set_printoptions(threshold=np.inf)

In [3]:
from load_data import load_file, separate_data_label, label_encoding

In [4]:
from text_cnn import TextCNN

In [5]:
from plotting import scatterplot

In [6]:
# Load classified Document
document1 = load_file("2007ChallengeTrainData.xml")
id_doc1, labels, words1 = separate_data_label(document1)

In [7]:
# Encoding words by one-hot encoding
max_document_length = max([len(x.split(" ")) for x in words1])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(words1)))

In [8]:
# One-hot-encoding label of each document
# There are 978 documents and 45 labels
y = label_encoding(labels)                                      # Encode data Label in DataFrame type
y = y.as_matrix()                                               # Convert DataFrame type to numpy array 

In [9]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [10]:
# Split train/test set
# TODO: This is very crude, should use cross-validation
# split data into training (80%) and test (20%)
x_train, x_dev = train_test_split(x_shuffled, train_size = 0.8)
y_train, y_dev = train_test_split(y_shuffled, train_size = 0.8)
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 1551
Train/Dev split: 782/196


In [11]:
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0
batch_size = 64
num_epochs = 200
evaluate_every = 100
checkpoint_every = 100

In [12]:
# Training
# ==================================================
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=True,
      log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=45,
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=filter_sizes,
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.histogram_summary("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.scalar_summary("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.merge_summary(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.scalar_summary("loss", cnn.loss)
        acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.merge_summary([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.merge_summary([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.train.SummaryWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.all_variables())

        # Store vocabulary embedded
        #dictionary.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.initialize_all_variables())

Writing to /home/ubuntu/workspace/tensorflow/thesis/Real/runs/1474624998



In [13]:
# Sort each data and label into one pair
# then shuffle to ensure randomness 
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [14]:
batches = batch_iter(
        list(zip(x_train, y_train)), batch_size, num_epochs)

In [15]:
def train_step(x_batch, y_batch):
    """
    A single training step
    """
    feed_dict = {
        cnn.input_x: x_batch,
        cnn.input_y: y_batch,
        cnn.dropout_keep_prob: dropout_keep_prob
    }
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    print("step {}, loss {:g}, acc {:g}".format(step, loss, accuracy))
    train_summary_writer.add_summary(summaries, step)

In [16]:
def dev_step(x_batch, y_batch, writer=None):
    """
    Evaluates model on a dev set
    """
    feed_dict = {
        cnn.input_x: x_batch,
        cnn.input_y: y_batch,
        cnn.dropout_keep_prob: 1.0
    }
    step, summaries, loss, accuracy, y_p = sess.run(
        [global_step, dev_summary_op, cnn.loss, cnn.accuracy, cnn.y_p],
        feed_dict)
    y_true = np.argmax(y_dev,1)
    precision = sk.metrics.precision_score(y_true, y_p)
    recall = sk.metrics.recall_score(y_true, y_p)
    f1_score = sk.metrics.f1_score(y_true, y_p)
    print("step {}, loss {:g}, acc {:g}, precison {}, recall {}, f1_score {}.".format(step, loss, accuracy, precision, recall, f1_score))

In [17]:
for batch in batches:
    x_batch, y_batch = zip(*batch)
    train_step(x_batch, y_batch)
    current_step = tf.train.global_step(sess, global_step)
    if current_step % evaluate_every == 0:
        print("\nEvaluation:")
        dev_step(x_dev, y_dev, writer=dev_summary_writer)
        print("")

step 1, loss 11.729, acc 0.03125
step 2, loss 11.6866, acc 0.03125
step 3, loss 10.0553, acc 0.03125
step 4, loss 8.00012, acc 0.078125
step 5, loss 7.73434, acc 0.09375
step 6, loss 7.16628, acc 0.0625
step 7, loss 8.47356, acc 0.09375
step 8, loss 6.73954, acc 0.109375
step 9, loss 7.83743, acc 0.046875
step 10, loss 7.63223, acc 0.0625
step 11, loss 7.29401, acc 0.15625
step 12, loss 8.91101, acc 0.15625
step 13, loss 7.00955, acc 0.142857
step 14, loss 7.45694, acc 0.125
step 15, loss 8.72629, acc 0.109375
step 16, loss 7.1624, acc 0.09375
step 17, loss 7.65224, acc 0.109375
step 18, loss 7.25049, acc 0.109375
step 19, loss 6.82871, acc 0.09375
step 20, loss 7.06514, acc 0.125
step 21, loss 6.52199, acc 0.15625
step 22, loss 7.99467, acc 0.125
step 23, loss 7.53479, acc 0.140625
step 24, loss 6.13352, acc 0.171875
step 25, loss 7.03645, acc 0.109375
step 26, loss 7.87081, acc 0.0714286
step 27, loss 7.75492, acc 0.109375
step 28, loss 5.92623, acc 0.171875
step 29, loss 6.23983, ac

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None wil

step 100, loss 5.52084, acc 0.147959, precison 0.0694134796776, recall 0.147959183673, f1_score 0.0754224270353.

step 101, loss 5.66475, acc 0.21875
step 102, loss 4.61649, acc 0.3125
step 103, loss 4.96718, acc 0.3125
step 104, loss 4.4648, acc 0.357143
step 105, loss 4.79152, acc 0.25
step 106, loss 5.17589, acc 0.265625
step 107, loss 4.12967, acc 0.328125
step 108, loss 5.31896, acc 0.265625
step 109, loss 6.37365, acc 0.15625
step 110, loss 5.59788, acc 0.234375
step 111, loss 5.27519, acc 0.171875
step 112, loss 5.54012, acc 0.140625
step 113, loss 5.78326, acc 0.25
step 114, loss 5.11888, acc 0.25
step 115, loss 4.54182, acc 0.25
step 116, loss 5.53017, acc 0.28125
step 117, loss 3.1244, acc 0.285714
step 118, loss 5.03389, acc 0.203125
step 119, loss 5.81637, acc 0.25
step 120, loss 4.81936, acc 0.25
step 121, loss 4.49979, acc 0.28125
step 122, loss 4.99756, acc 0.171875
step 123, loss 4.2555, acc 0.328125
step 124, loss 5.62062, acc 0.25
step 125, loss 4.8206, acc 0.3125
ste

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sampl

step 200, loss 8.78048, acc 0.142857, precison 0.0768164885002, recall 0.142857142857, f1_score 0.0752513756973.

step 201, loss 6.71003, acc 0.1875
step 202, loss 6.55135, acc 0.3125
step 203, loss 6.23669, acc 0.3125
step 204, loss 7.46163, acc 0.203125
step 205, loss 7.03062, acc 0.28125
step 206, loss 6.15906, acc 0.328125
step 207, loss 7.38815, acc 0.203125
step 208, loss 7.89977, acc 0.571429
step 209, loss 6.4608, acc 0.25
step 210, loss 7.61431, acc 0.34375
step 211, loss 7.37469, acc 0.234375
step 212, loss 5.39331, acc 0.15625
step 213, loss 7.98626, acc 0.3125
step 214, loss 4.99556, acc 0.328125
step 215, loss 6.32854, acc 0.203125
step 216, loss 7.53699, acc 0.234375
step 217, loss 7.97011, acc 0.234375
step 218, loss 8.67534, acc 0.21875
step 219, loss 7.85673, acc 0.265625
step 220, loss 7.52277, acc 0.28125
step 221, loss 13.5907, acc 0
step 222, loss 9.23382, acc 0.203125
step 223, loss 9.26068, acc 0.25
step 224, loss 10.3793, acc 0.203125
step 225, loss 9.58157, acc

...

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sampl

step 2400, loss 192387, acc 0.112245, precison 0.0134205856256, recall 0.112244897959, f1_score 0.023974638399.

step 2401, loss 14114.9, acc 0.0625
step 2402, loss 10380.5, acc 0.109375
step 2403, loss 12004.8, acc 0.078125
step 2404, loss 14953.2, acc 0.078125
step 2405, loss 8494.36, acc 0
step 2406, loss 20987.2, acc 0.0625
step 2407, loss 18618.7, acc 0.171875
step 2408, loss 15948.5, acc 0.09375
step 2409, loss 15540.3, acc 0.046875
step 2410, loss 11837.6, acc 0.046875
step 2411, loss 18136.1, acc 0.078125
step 2412, loss 10997.4, acc 0.140625
step 2413, loss 11561, acc 0.171875
step 2414, loss 11335.4, acc 0.125
step 2415, loss 12304.6, acc 0.140625
step 2416, loss 16675.9, acc 0.078125
step 2417, loss 16439.4, acc 0.171875
step 2418, loss 9676.21, acc 0.0714286
step 2419, loss 11897.1, acc 0.078125
step 2420, loss 19866.4, acc 0.0625
step 2421, loss 11072.5, acc 0.0625
step 2422, loss 17346.2, acc 0.125
step 2423, loss 16161, acc 0.140625
step 2424, loss 14178.9, acc 0.125
ste

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sampl

step 2500, loss 212866, acc 0.163265, precison 0.0372280780444, recall 0.163265306122, f1_score 0.0568123890383.

step 2501, loss 14746.7, acc 0.078125
step 2502, loss 20310.7, acc 0.0625
step 2503, loss 18198.7, acc 0.15625
step 2504, loss 13170.5, acc 0.078125
step 2505, loss 16548.2, acc 0.125
step 2506, loss 15954.3, acc 0.046875
step 2507, loss 13537.2, acc 0.09375
step 2508, loss 9131.97, acc 0.140625
step 2509, loss 8456, acc 0.142857
step 2510, loss 15596.6, acc 0.09375
step 2511, loss 13242.7, acc 0.0625
step 2512, loss 14231.9, acc 0.078125
step 2513, loss 13952, acc 0.09375
step 2514, loss 11577.9, acc 0.09375
step 2515, loss 14743.4, acc 0.078125
step 2516, loss 14666.2, acc 0.109375
step 2517, loss 13954.1, acc 0.0625
step 2518, loss 19236.8, acc 0.109375
step 2519, loss 14802.2, acc 0.046875
step 2520, loss 14170.4, acc 0.046875
step 2521, loss 15232.8, acc 0.0625
step 2522, loss 16295.4, acc 0.214286
step 2523, loss 17168.2, acc 0.0625
step 2524, loss 10418.8, acc 0.0156

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sampl

In [ ]:
pred = multilayer_perceptron(x, weights, biases)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
init = tf.initialize_all_variables()
sess.run(init)
for epoch in xrange(150):
        for i in xrange(total_batch):
                train_step.run(feed_dict = {x: train_arrays, y: train_labels})
                avg_cost += sess.run(cost, feed_dict={x: train_arrays, y: train_labels})/total_batch         
        if epoch % display_step == 0:
                print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

#metrics
y_p = tf.argmax(pred, 1)
val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x:test_arrays, y:test_label})

print "validation accuracy:", val_accuracy
y_true = np.argmax(test_label,1)
print "Precision", sk.metrics.precision_score(y_true, y_pred)
print "Recall", sk.metrics.recall_score(y_true, y_pred)
print "f1_score", sk.metrics.f1_score(y_true, y_pred)
print "confusion_matrix"
print sk.metrics.confusion_matrix(y_true, y_pred)
fpr, tpr, tresholds = sk.metrics.roc_curve(y_true, y_pred)

### Using Normal Training

In [14]:
batches = batch_iter(
        list(zip(x_train, y_train)), batch_size, num_epochs)

In [15]:
for batch in batches:
    x_batch, y_batch = zip(*batch)
    train_step(x_batch, y_batch)
    current_step = tf.train.global_step(sess, global_step)
    if current_step % evaluate_every == 0:
        print("\nEvaluation:")
        dev_step(x_dev, y_dev, writer=dev_summary_writer)
        print("")
        if current_step % checkpoint_every == 0:
            path = saver.save(sess, checkpoint_prefix, global_step=current_step)
            print("Saved model checkpoint to {}\n".format(path))

2016-09-11T09:20:38.721420: step 1, loss 9.62068, acc 0.078125
2016-09-11T09:20:39.893012: step 2, loss 10.6179, acc 0.046875
2016-09-11T09:20:40.907889: step 3, loss 8.47935, acc 0.046875
2016-09-11T09:20:41.906054: step 4, loss 8.49473, acc 0.09375
2016-09-11T09:20:42.918021: step 5, loss 7.4866, acc 0.09375
2016-09-11T09:20:44.104501: step 6, loss 7.46709, acc 0.078125
2016-09-11T09:20:45.104881: step 7, loss 8.63967, acc 0.09375
2016-09-11T09:20:46.109960: step 8, loss 6.89962, acc 0.109375
2016-09-11T09:20:47.260477: step 9, loss 7.13895, acc 0.1875
2016-09-11T09:20:48.279823: step 10, loss 7.81613, acc 0.09375
2016-09-11T09:20:49.291971: step 11, loss 8.04364, acc 0.171875
2016-09-11T09:20:50.271658: step 12, loss 8.34044, acc 0.171875
2016-09-11T09:20:50.506905: step 13, loss 8.10609, acc 0
2016-09-11T09:20:51.515571: step 14, loss 7.09213, acc 0.15625
2016-09-11T09:20:52.542369: step 15, loss 7.51027, acc 0.09375
2016-09-11T09:20:53.516869: step 16, loss 7.71323, acc 0.171875
2

### Apply Cross Validation

In [17]:
## Create 10-Fold validation
folds = KFold(n=len(words1), n_folds=10, shuffle=True)

In [ ]:
def folds_values(data, folds_values):
    data = np.array(data)
    for train_index, test_index in folds_values:
        folds_train = data[train_index]
        folds_dev = data[test_index]
        yield folds_train,folds_dev

In [ ]:
return_values = folds_values(list(zip(x_shuffled, y_shuffled)), folds)

In [ ]:
for train_index, test_index in return_values:
    #x_train, y_train = zip(*train_index)
    x_test, y_test = zip(*test_index)
    batches = batch_iter(
        train_index, batch_size, num_epochs)
    for batch in batches:
        x_batch, y_batch = zip(*batch)
        train_step(x_batch, y_batch)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % evaluate_every == 0:
            print("\nEvaluation:")
            dev_step(x_test, y_test, writer=dev_summary_writer)
            print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))